# 16. Классы nn.Linear и nn.Module

In [1]:
import torch
import numpy as np
from random import randint
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### создание НС через класс

In [2]:
class NetGirl(nn.Module):
    def __init__(self, input_dim, num_hidden, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, num_hidden)
        self.layer2 = nn.Linear(num_hidden, output_dim)
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = F.tanh(x)
        x = self.layer2(x)
        x = F.tanh(x)
        return x

In [3]:
model = NetGirl(3, 2, 1)

print(model)

NetGirl(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)


In [4]:
gen_p = model.parameters() #генератор параметров 
print(list(gen_p))

[Parameter containing:
tensor([[-0.2633, -0.1873, -0.5740],
        [ 0.1524,  0.2833,  0.3643]], requires_grad=True), Parameter containing:
tensor([0.0059, 0.3886], requires_grad=True), Parameter containing:
tensor([[ 0.3591, -0.5002]], requires_grad=True), Parameter containing:
tensor([0.6868], requires_grad=True)]


In [ ]:
x_train = torch.FloatTensor([(-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
                             (1, -1, -1), (1, -1, 1), (1, 1, -1), (1, 1, 1)])

y_train = torch.FloatTensor([-1, 1, -1, 1, -1, 1, -1, -1])

total = len(y_train)

#оптимизатор 
optimizer = optim.RMSprop(params=model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()


#обучение модели
model.train()

NetGirl(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

In [6]:
#стохастический градиентный спуск
for _ in range(1000):
    k = randint(0, total - 1)
    y = model(x_train[k])
    y = y.squeeze()
    loss = loss_func(y, y_train[k])
    
    #шаг градиентного спуска
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [7]:
model.eval()

#отключение локального градиента
#model.requires_grad_(False) 

#включение локального градиента
#model.requires_grad_(True) 

NetGirl(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

In [8]:
for x, d in zip(x_train, y_train):
    with torch.no_grad(): # отключение локального градиента
        y = model(x)
        print(f'Выходное значение НС: {y.data} => {d}')

Выходное значение НС: tensor([-0.9812]) => -1.0
Выходное значение НС: tensor([0.9989]) => 1.0
Выходное значение НС: tensor([-0.9987]) => -1.0
Выходное значение НС: tensor([0.9824]) => 1.0
Выходное значение НС: tensor([-0.9987]) => -1.0
Выходное значение НС: tensor([0.9820]) => 1.0
Выходное значение НС: tensor([-0.9990]) => -1.0
Выходное значение НС: tensor([-0.9732]) => -1.0


#### создание НС напрямую

In [9]:
def forward(inp, l1: nn.Linear, l2: nn.Linear):
    u1 = l1.forward(inp)
    s1 = F.tanh(u1)
    
    u2 = l2.forward(s1)
    s2 = F.tanh(u2)
    return s2

In [10]:
layer1 = nn.Linear(in_features=3, out_features=2)
layer2 = nn.Linear(in_features=2, out_features=1) # layer2 = nn.Linear(2, 1)

print(layer1.weight)
print(layer1.bias)

Parameter containing:
tensor([[-0.5300, -0.0917,  0.5600],
        [ 0.0619, -0.1564, -0.3987]], requires_grad=True)
Parameter containing:
tensor([-0.3871,  0.2471], requires_grad=True)


In [11]:
layer1.weight.data = torch.tensor([[0.7402, 0.6008, -1.3340], [0.2098, 0.4537, -0.7692]])
layer1.bias.data = torch.tensor([0.5505, 0.3719])

layer2.weight.data = torch.tensor([[-2.0719, -0.9485]])
layer2.bias.data = torch.tensor([-0.1461])

In [12]:
x = torch.FloatTensor([1, -1, 1])
y = forward(x, layer1, layer2)

print(y.data)

tensor([0.9165])
